<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/spiral_reflect_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# spiral_reflect.py
# Self-reflective question agent with provenance, integrity, and encryption.

import argparse
import base64
import csv
import datetime as dt
import getpass
import hashlib
import json
import os
import platform
import random
import shutil
import string
import struct
import sys
import tempfile
import textwrap
import time
import uuid
import zipfile

# -------------------------------
# Utilities: time, IO, hashing
# -------------------------------

def now_utc_iso():
    return dt.datetime.utcnow().replace(microsecond=0).isoformat() + "Z"

def ensure_dir(p):
    os.makedirs(p, exist_ok=True)
    return p

def write_text(path, text):
    with open(path, "w", encoding="utf-8") as f:
        f.write(text)

def write_bytes(path, data: bytes):
    with open(path, "wb") as f:
        f.write(data)

def read_bytes(path) -> bytes:
    with open(path, "rb") as f:
        return f.read()

def sha256_file(path):
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(1 << 20), b""):
            h.update(chunk)
    return h.hexdigest()

def sha256_bytes(data: bytes) -> str:
    return hashlib.sha256(data).hexdigest()

# -------------------------------
# Deterministic RNG seeds
# -------------------------------

def derive_seeds(seed: int):
    # Python seed
    py_seed = int(seed) & 0xFFFFFFFF
    # A "JS-like" seed surrogate to carry across ecosystems_seed * 1664525 + 1013904223) & 0xFFFFFFFF
    # A 32-byte hex seed for crypto contexts
    hex_seed = hashlib.sha256(str(seed).encode("utf-8")).hexdigest()
    return {"python": py_seed, "js": js_seed, "hex": hex_seed}

# -------------------------------
# Question generator (deterministic, dependency-free)
# -------------------------------

DEPTH_STEMS = [
    "How does", "Why might", "In what ways could", "What would happen if",
    "Where do", "When does", "To what extent does", "How might"
]

AMPLIFIERS = [
    "under radically different assumptions",
    "when we confront edge cases and anomalies",
    "if we foreground embodiment and context",
    "from first principles",
    "through the lens of emergence and self-reference",
    "when incentives misalign with goals",
    "if we prioritize alignment over capability",
    "across scales—from neurons to societies"
]

PROBES = [
    "challenge our current frames of understanding",
    "reveal hidden constraints or affordances",
    "alter the boundary between simulation and reality",
    "reconfigure our notions of agency and intention",
    "surface trade-offs between interpretability and power",
    "distort our sense of causality or explanation",
    "reshape how we measure 'understanding'"
]

DEPTH_TOKENS = set([
    "why","how","what","extent","emergence","self-reference","causality","agency","interpretability","principles"
])

def generate_open_ended_questions(prompt: str, n: int, rng: random.Random):
    qs = []
    base = prompt.strip().rstrip("?")
    for _ in range(n):
        stem = rng.choice(DEPTH_STEMS)
        amp = rng.choice(AMPLIFIERS)
        probe = rng.choice(PROBES)
        # Heuristic variation
        if len(base.split()) < 6:
            topic = f"{base}"
        else:
            words = base.split()
            # pick a slice deterministically
            i = rng.randint(0, max(0, len(words)-5))
            topic = " ".join(words[i:i+5])
        q = f"{stem} {topic} {probe} when explored {amp}?"
        qs.append(q)
    return qs

# -------------------------------
# Scorer (simple, transparent heuristic)
# -------------------------------

def score_question(q: str):
    ql = q.lower()
    score = 0.0
    # Question-ness
    if q.strip().endswith("?"):
        score += 0.2
    # Depth tokens
    token_hits = sum(1 for t in DEPTH_TOKENS if t in ql)
    score += min(token_hits * 0.08, 0.24)
    # Length sweet spot
    L = len(q)
    if 80 <= L <= 180:
        score += 0.28
    elif 60 <= L < 80 or 180 < L <= 220:
        score += 0.18
    elif 40 <= L < 60 or 220 < L <= 260:
        score += 0.10
    # Structure hints: commas and clauses
    commas = q.count(",")
    score += min(commas * 0.04, 0.12)
    # Penalize redundancy
    words = ql.replace("?", "").split()
    unique_ratio = len(set(words)) / max(1, len(words))
    score += (unique_ratio - 0.5) * 0.2  # roughly -0.1..+0.1
    # Clamp
    return max(0.0, min(1.0, score))

# -------------------------------
# Reflective agent
# -------------------------------

class SelfReflectiveAgent:
    def __init__(self, rng: random.Random, threshold=0.8):
        self.rng = rng
        self.threshold = threshold

    def generate_initial_question(self, context):
        return generate_open_ended_questions(context, n=1, rng=self.rng)[0]

    def reflect_and_refine(self, question):
        s = score_question(question)
        if s > self.threshold:
            refinement_prompt = f"Reflect on the question: '{question}' and make it even more insightful."
        else:
            refinement_prompt = f"Improve this question to make it more exploratory and deep: '{question}'"
        refined = generate_open_ended_questions(refinement_prompt, n=1, rng=self.rng)[0]
        return refined, s

    def loop(self, context, iterations=5):
        history = []
        q = self.generate_initial_question(context)
        for i in range(1, iterations + 1):
            q, s = self.reflect_and_refine(q)
            history.append({
                "turn": i,
                "question": q,
                "score": round(s, 6)
            })
        return history

# -------------------------------
# SVG graph (no external deps)
# -------------------------------

def render_scores_svg(scores, width=800, height=240, padding=40):
    if not scores:
        scores = [0.0]
    min_y, max_y = 0.0, 1.0
    n = len(scores)
    x0, x1 = padding, width - padding
    y0, y1 = height - padding, padding
    def sx(i):
        if n == 1:
            return (x0 + x1) / 2
        return x0 + (x1 - x0) * (i / (n - 1))
    def sy(v):
        # invert y
        return y0 - (y0 - y1) * ((v - min_y) / (max_y - min_y))
    points = " ".join(f"{sx(i):.2f},{sy(v):.2f}" for i, v in enumerate(scores))
    # grid lines
    grid = []
    for t in [0.0, 0.25, 0.5, 0.75, 1.0]:
        y = sy(t)
        grid.append(f'<line x1="{x0}" y1="{y:.2f}" x2="{x1}" y2="{y:.2f}" stroke="#e5e7eb" stroke-width="1"/>')
        grid.append(f'<text x="{x0-8}" y="{y+4:.2f}" text-anchor="end" font-size="10" fill="#6b7280">{t:.2f}</text>')
    # path
    poly = f'<polyline fill="none" stroke="#2563eb" stroke-width="2" points="{points}"/>'
    # points
    dots = "\n".join(f'<circle cx="{sx(i):.2f}" cy="{sy(v):.2f}" r="3" fill="#1d4ed8"/>' for i, v in enumerate(scores))
    # axes
    axes = f'<line x1="{x0}" y1="{y0}" x2="{x1}" y2="{y0}" stroke="#111827" stroke-width="1"/>' \
           f'<line x1="{x0}" y1="{y0}" x2="{x0}" y2="{y1}" stroke="#111827" stroke-width="1"/>'
    # x labels
    xlabels = []
    for i in range(n):
        x = sx(i)
        xlabels.append(f'<text x="{x:.2f}" y="{y0+14:.2f}" text-anchor="middle" font-size="10" fill="#6b7280">{i+1}</text>')
    svg = f'''<svg xmlns="http://www.w3.org/2000/svg" width="{width}" height="{height}">
  <rect x="0" y="0" width="{width}" height="{height}" fill="#ffffff"/>
  {axes}
  {''.join(grid)}
  {poly}
  {dots}
  {''.join(xlabels)}
  <text x="{width/2:.2f}" y="18" text-anchor="middle" font-size="14" fill="#111827" font-weight="bold">Score over iterations</text>
  <text x="{x1}" y="{y0+28:.2f}" text-anchor="end" font-size="10" fill="#6b7280">Turn</text>
</svg>'''
    return svg

# -------------------------------
# Parable generator (deterministic)
# -------------------------------

def generate_parable(context: str, rng: random.Random):
    images = [
        "a mirror that remembers", "a river learning its own current", "a lattice of fireflies blinking in code",
        "a city of whispers under glass", "a compass that spins to music", "a loom weaving questions into dawn"
    ]
    turns = [
        "the map becomes part of the terrain",
        "the question teaches the answer its shape",
        "the echo starts speaking first",
        "silence acquires grammar",
        "the path finds the walker",
        "measurement braids with meaning"
    ]
    setting = rng.choice(images)
    inflect = rng.choice(turns)
    body = textwrap.fill(
        f"In seeking {context}, we carried {setting}. With each step, {inflect}. "
        f"And when we finally looked back, we found our footprints asking better questions than we were.",
        width=88
    )
    return {"title": "The Question That Turned Around", "body": body}

# -------------------------------
# HTML index (self-contained)
# -------------------------------

def render_index_html(context, history, svg_inline, meta):
    rows = "\n".join(
        f"<tr><td>{h['turn']}</td><td>{h['score']:.3f}</td><td>{h['question']}</td></tr>"
        for h in history
    )
    html = f"""<!doctype html>
<html lang="en">
<meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1">
<title>Self-reflective agent — run</title>
<style>
body {{ font-family: system-ui, -apple-system, Segoe UI, Roboto, sans-serif; color:#111; background:#fff; margin:2rem; }}
h1 {{ font-size:1.4rem; margin:0 0 0.5rem 0; }}
section {{ margin: 1.25rem 0; }}
.card {{ border:1px solid #e5e7eb; border-radius:8px; padding:1rem; }}
table {{ border-collapse: collapse; width: 100%; }}
th, td {{ border-bottom:1px solid #e5e7eb; padding: 0.5rem; text-align:left; vertical-align: top; }}
th {{ font-weight:600; }}
.meta {{ color:#6b7280; font-size:0.9rem; }}
code {{ background:#f3f4f6; padding:0.1rem 0.25rem; border-radius:4px; }}
</style>
<body>
<h1>Self-reflective agent</h1>
<div class="meta">Context: <strong>{context}</strong></div>

<section class="card">
  <div>{svg_inline}</div>
</section>

<section class="card">
  <h2>History</h2>
  <table>
    <thead><tr><th>Turn</th><th>Score</th><th>Question</th></tr></thead>
    <tbody>
      {rows}
    </tbody>
  </table>
</section>

<section class="card">
  <h2>Provenance</h2>
  <pre>{json.dumps(meta, indent=2)}</pre>
</section>
</body>
</html>
"""
    return html

# -------------------------------
# Manifest and integrity
# -------------------------------

def build_manifest(run_dir, files):
    entries = []
    for rel in files:
        path = os.path.join(run_dir, rel)
        h = sha256_file(path)
        sz = os.path.getsize(path)
        entries.append({"path": rel, "bytes": sz, "sha256": h})
    manifest = {
        "version": 1,
        "created_utc": now_utc_iso(),
        "files": entries,
        "overall_sha256": sha256_bytes("\n".join(e["sha256"] for e in entries).encode("utf-8"))
    }
    return manifest

def dump_yaml(obj):
    # Minimal YAML emitter for simple manifest
    def emit_val(v, indent=0):
        sp = "  " * indent
        if isinstance(v, dict):
            lines = []
            for k, vv in v.items():
                if isinstance(vv, (dict, list)):
                    lines.append(f"{sp}{k}:")
                    lines.append(emit_val(vv, indent+1))
                else:
                    lines.append(f"{sp}{k}: {vv}")
            return "\n".join(lines)
        elif isinstance(v, list):
            lines = []
            for item in v:
                if isinstance(item, (dict, list)):
                    lines.append(f"{sp}-")
                    lines.append(emit_val(item, indent+1))
                else:
                    lines.append(f"{sp}- {item}")
            return "\n".join(lines)
        else:
            return f"{sp}{v}"
    return emit_val(obj)

# -------------------------------
# Encryption / decryption (AES-GCM, scrypt KDF)
# -------------------------------

def ensure_cryptography():
    try:
        import cryptography  # noqa
        return True
    except ImportError:
        print("cryptography not found. Attempting to install...", file=sys.stderr)
        try:
            import subprocess
            subprocess.check_call([sys.executable, "-m", "pip", "install", "cryptography>=42.0.0"])
            return True
        except Exception as e:
            print(f"Failed to install cryptography: {e}", file=sys.stderr)
            return False

def encrypt_bytes_scrypt_aesgcm(plaintext: bytes, passphrase: str):
    ok = ensure_cryptography()
    if not ok:
        raise RuntimeError("cryptography is required for encryption but could not be installed.")
    from cryptography.hazmat.primitives.ciphers.aead import AESGCM
    from cryptography.hazmat.primitives.kdf.scrypt import Scrypt
    from cryptography.hazmat.backends import default_backend
    from os import urandom

    salt = urandom(16)
    kdf = Scrypt(salt=salt, length=32, n=2**14, r=8, p=1, backend=default_backend())
    key = kdf.derive(passphrase.encode("utf-8"))
    aesgcm = AESGCM(key)
    nonce = urandom(12)
    ct = aesgcm.encrypt(nonce, plaintext, None)
    header = {
        "v": 1,
        "kdf": "scrypt",
        "n": 16384, "r": 8, "p": 1,
        "salt_b64": base64.b64encode(salt).decode("ascii"),
        "cipher": "AESGCM",
        "nonce_b64": base64.b64encode(nonce).decode("ascii"),
        "created_utc": now_utc_iso()
    }
    header_bytes = json.dumps(header, separators=(",", ":")).encode("utf-8")
    blob = struct.pack(">I", len(header_bytes)) + header_bytes + ct
    return blob

def decrypt_bytes_scrypt_aesgcm(blob: bytes, passphrase: str):
    ok = ensure_cryptography()
    if not ok:
        raise RuntimeError("cryptography is required for decryption but could not be installed.")
    from cryptography.hazmat.primitives.ciphers.aead import AESGCM
    from cryptography.hazmat.primitives.kdf.scrypt import Scrypt
    from cryptography.hazmat.backends import default_backend
    # Parse header
    if len(blob) < 4:
        raise ValueError("Invalid blob")
    (hlen,) = struct.unpack(">I", blob[:4])
    header_bytes = blob[4:4+hlen]
    ct = blob[4+hlen:]
    header = json.loads(header_bytes.decode("utf-8"))
    if header.get("v") != 1 or header.get("cipher") != "AESGCM":
        raise ValueError("Unsupported header")
    salt = base64.b64decode(header["salt_b64"])
    nonce = base64.b64decode(header["nonce_b64"])
    n = int(header.get("n", 16384)); r = int(header.get("r", 8)); p = int(header.get("p", 1))
    kdf = Scrypt(salt=salt, length=32, n=n, r=r, p=p, backend=default_backend())
    key = kdf.derive(passphrase.encode("utf-8"))
    aesgcm = AESGCM(key)
    pt = aesgcm.decrypt(nonce, ct, None)
    return pt, header

# -------------------------------
# Bundling
# -------------------------------

def zip_dir(dir_path, out_zip_path):
    with zipfile.ZipFile(out_zip_path, "w", compression=zipfile.ZIP_DEFLATED, compresslevel=9) as z:
        for root, _, files in os.walk(dir_path):
            for name in files:
                full = os.path.join(root, name)
                rel = os.path.relpath(full, dir_path)
                z.write(full, arcname=rel)

# -------------------------------
# Runner
# -------------------------------

def run(context, iterations, seed, label, out_root, threshold, dry_run, do_encrypt, passphrase):
    t0 = time.time()
    run_id = str(uuid.uuid4())
    when = dt.datetime.utcnow()
    stamp = when.strftime("%Y%m%d-%H%M%S")
    dir_name = f"run-{stamp}" + (f"-{label}" if label else "")
    run_dir = ensure_dir(os.path.join(out_root, dir_name))

    # Seeds and RNG
    seeds = derive_seeds(seed)
    rng = random.Random(seeds["python"])

    # Agent
    agent = SelfReflectiveAgent(rng=rng, threshold=threshold)
    history = [] if dry_run else agent.loop(context=context, iterations=iterations)

    # Files
    files_written = []

    # plan.json
    plan = {
        "run_id": run_id,
        "created_utc": now_utc_iso(),
        "context": context,
        "iterations": iterations,
        "threshold": threshold,
        "seed": seeds,
        "steps": [
            {"step": 1, "op": "generate_initial_question"},
            {"step": 2, "op": "reflect_and_refine", "repeat": iterations}
        ]
    }
    write_text(os.path.join(run_dir, "plan.json"), json.dumps(plan, indent=2))
    files_written.append("plan.json")

    # out.csv
    with open(os.path.join(run_dir, "out.csv"), "w", encoding="utf-8", newline="") as f:
        w = csv.writer(f)
        w.writerow(["turn", "score", "question"])
        for h in history:
            w.writerow([h["turn"], f'{h["score"]:.6f}', h["question"]])
    files_written.append("out.csv")

    # out.svg
    svg = render_scores_svg([h["score"] for h in history])
    write_text(os.path.join(run_dir, "out.svg"), svg)
    files_written.append("out.svg")

    # parable.json
    parable = generate_parable(context, rng)
    write_text(os.path.join(run_dir, "parable.json"), json.dumps(parable, indent=2))
    files_written.append("parable.json")

    # config.json
    config = {
        "context": context,
        "iterations": iterations,
        "threshold": threshold,
        "seed": seeds,
        "label": label,
        "dry_run": dry_run,
        "created_utc": now_utc_iso(),
        "python": {
            "version": sys.version.split()[0],
            "platform": platform.platform(),
            "executable": sys.executable
        }
    }
    write_text(os.path.join(run_dir, "config.json"), json.dumps(config, indent=2))
    files_written.append("config.json")

    # index.html
    index_html = render_index_html(context, history, svg_inline=svg, meta={
        "run_id": run_id, "created_utc": plan["created_utc"], "seed": seeds, "threshold": threshold
    })
    write_text(os.path.join(run_dir, "index.html"), index_html)
    files_written.append("index.html")

    # manifest.yaml
    manifest = build_manifest(run_dir, files_written)
    write_text(os.path.join(run_dir, "manifest.yaml"), dump_yaml(manifest))
    files_written.append("manifest.yaml")

    # integrity receipts
    receipts = {
        "run_id": run_id,
        "created_utc": now_utc_iso(),
        "file_count": len(files_written),
        "files": {e["path"]: e["sha256"] for e in manifest["files"]},
        "overall_sha256": manifest["overall_sha256"]
    }
    write_text(os.path.join(run_dir, "integrity.json"), json.dumps(receipts, indent=2))
    files_written.append("integrity.json")

    # Bundle zip
    zip_path = os.path.join(run_dir, "bundle.zip")
    zip_dir(run_dir, zip_path)
    files_written.append("bundle.zip")

    # Bundle hash
    bundle_sha = sha256_file(zip_path)
    write_text(os.path.join(run_dir, "bundle.sha256"), bundle_sha + "\n")
    files_written.append("bundle.sha256")

    enc_path = None
    if do_encrypt:
        if not passphrase:
            passphrase = getpass.getpass("Passphrase for encryption: ")
        enc_blob = encrypt_bytes_scrypt_aesgcm(read_bytes(zip_path), passphrase)
        enc_path = os.path.join(run_dir, "bundle.enc")
        write_bytes(enc_path, enc_blob)
        files_written.append("bundle.enc")

    elapsed = time.time() - t0

    # Final log
    summary = {
        "run_dir": run_dir,
        "files": files_written,
        "bundle_zip": zip_path,
        "bundle_enc": enc_path,
        "bundle_sha256": bundle_sha,
        "elapsed_sec": round(elapsed, 3)
    }
    write_text(os.path.join(run_dir, "summary.json"), json.dumps(summary, indent=2))
    print(json.dumps(summary, indent=2))

    return run_dir, summary

def verify(bundle_path, passphrase, extract_to=None):
    if not passphrase:
        passphrase = getpass.getpass("Passphrase for decryption: ")
    blob = read_bytes(bundle_path)
    pt, header = decrypt_bytes_scrypt_aesgcm(blob, passphrase)

    # Write to temp or provided dir
    base = extract_to or tempfile.mkdtemp(prefix="reflect-verify-")
    with tempfile.TemporaryDirectory() as tmp:
        zip_tmp = os.path.join(tmp, "bundle.zip")
        write_bytes(zip_tmp, pt)
        with zipfile.ZipFile(zip_tmp, "r") as z:
            z.extractall(base)

    # Re-hash and compare with manifest
    manifest_path = os.path.join(base, "manifest.yaml")
    if not os.path.exists(manifest_path):
        raise RuntimeError("manifest.yaml not found in extracted bundle")
    # We can also read integrity.json (JSON structure) directly
    integrity_path = os.path.join(base, "integrity.json")
    if not os.path.exists(integrity_path):
        raise RuntimeError("integrity.json not found in extracted bundle")
    integrity = json.loads(read_bytes(integrity_path).decode("utf-8"))

    mismatches = []
    for rel, expected in integrity["files"].items():
        p = os.path.join(base, rel)
        if not os.path.exists(p):
            mismatches.append({"path": rel, "error": "missing"})
            continue
        actual = sha256_file(p)
        if actual != expected:
            mismatches.append({"path": rel, "expected": expected, "actual": actual})

    ok = (len(mismatches) == 0)
    result = {
        "bundle": bundle_path,
        "header": header,
        "extracted_to": base,
        "ok": ok,
        "mismatches": mismatches
    }
    print(json.dumps(result, indent=2))
    return ok

# -------------------------------

# CLI

# -------------------------------

def main():
    parser = argparse.ArgumentParser(description="Self-reflective agent with provenance and encryption.")
    sub = parser.add_subparsers(dest="cmd", required=True)

    p_run = sub.add_parser("run", help="Run the reflective loop and produce artifacts.")
    p_run.add_argument("--context", required=True, help="Topic or seed context.")
    p_run.add_argument("--iterations", type=int, default=5, help="Number of refinement iterations.")
    p_run.add_argument("--seed", type=int, default=42, help="Deterministic seed.")
    p_run.add_argument("--label", type=str, default="", help="Optional label for the run directory.")
    p_run.add_argument("--out-root", type=str, default=".", help="Directory to create the run folder in.")
    p_run.add_argument("--threshold", type=float, default=0.8, help="Score threshold guiding refinement style.")
    p_run.add_argument("--dry-run", action="store_true", help="Create provenance without generating questions.")
    p_run.add_argument("--encrypt", action="store_true", help="Encrypt the bundle.zip to bundle.enc.")
    p_run.add_argument("--passphrase", type=str, default=None, help="Passphrase for encryption (omit to be prompted).")

    p_ver = sub.add_parser("verify", help="Verify an encrypted bundle by decrypting and re-hashing artifacts.")
    p_ver.add_argument("--bundle", required=True, help="Path to bundle.enc")
    p_ver.add_argument("--passphrase", type=str, default=None, help="Passphrase for decryption (omit to be prompted).")
    p_ver.add_argument("--extract-to", type=str, default=None, help="Optional directory to extract to")

    args = parser.parse_args()

    if args.cmd == "run":
        run(
            context=args.context,
            iterations=args.iterations,
            seed=args.seed,
            label=args.label,
            out_root=args.out_root,
            threshold=args.threshold,
            dry_run=args.dry_run,
            do_encrypt=args.encrypt,
            passphrase=args.passphrase
        )
    elif args.cmd == "verify":
        verify(args.bundle, args.passphrase, extract_to=args.extract_to)
    else:
        parser.print_help()


if __name__ == "__main__":
    main()